### Тестирование и описание конструкций LinearParametrization
В Этом файле приведены примеры основных блоков линеаризации:
1. Блок работы с характеристиками CH
2. Блок работы с характеристиками PWL

Чуть позже будет подготовлен пример с характеристиками CH_N (нейронные сети)

In [20]:
# Импорт модулей и подключение библиотек
import pandas as pd         # Панадас
import numpy as np          
from pyomo.environ import * # Библитека оптимизации
from LinearParametrization import *  # Модуль параметризации

# Считывание данных, на основании которых выполняется параметризация

In [25]:
df = pd.read_excel('DataMEI.xlsx')
df

,Qп,Qт,N,Q0
0,0,0,23.828635,67.105800
1,0,0,27.147921,73.242768
2,0,0,30.259031,79.088371
3,0,0,34.590692,87.168812
4,0,0,41.663221,100.216463
...,...,...,...,...
362,140,10,61.449816,87.312417
363,140,20,52.599275,63.309000
364,140,20,54.982477,68.017888
365,140,20,57.553562,73.094784


# Тестирование конструкции CH c дополнительными внешними ограничениями

In [38]:
# Задаем шкалу времени (24 часа), для которой будут проводиться расчеты
m = ConcreteModel()
nt=24
t = range(0, nt)
m.T1 = CH_Block(t, df)

In [39]:
# Задаем целевую функцию. Основная задача - максимизировать разницу дохода и расхода
Dohod = sum((m.T1.Vars[i,'Q0']) for i in t)
m.O1 = Objective(expr=Dohod , sense=maximize)

Ограничение именованное. Имя должно соответствовать имени одной из размерностей CH

In [35]:
#  Задание внешнего оганичения
Qp=df['Qп']
m.T1=ext_vars(m.T1,Qп=Qp)

['Qп']


In [41]:
# Используем наиболее быстрый решатель оптимизации Guroby Optimization для расчета нашей оптимизационной модели
gurobi = SolverFactory('gurobi')

# Смотрим время расчета и статус выполнения расчета модели
status = gurobi.solve(m)
print(status)


Problem: 
- Name: x98
  Lower bound: 3807.392219638474
  Upper bound: 3807.392219638474
  Number of objectives: 1
  Number of constraints: 1273
  Number of variables: 98
  Number of binary variables: 1
  Number of integer variables: 1
  Number of continuous variables: 97
  Number of nonzeros: 4873
  Sense: maximize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.05900001525878906
  Error rc: 0
  Time: 0.4621927738189697
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [42]:
m.T1.Vars.pprint()

Vars : Size=96, Index=T1.Vars_index
    Key        : Lower : Value              : Upper : Fixed : Stale : Domain
      (0, 'N') :  None :  75.03967027081069 :  None : False : False :  Reals
     (0, 'Q0') :  None : 158.64134248493644 :  None : False : False :  Reals
     (0, 'Qп') :  None : 20.000010451259246 :  None : False : False :  Reals
     (0, 'Qт') :  None :             0.0001 :  None : False : False :  Reals
      (1, 'N') :  None :  75.03967027081069 :  None : False : False :  Reals
     (1, 'Q0') :  None : 158.64134248493644 :  None : False : False :  Reals
     (1, 'Qп') :  None : 20.000010451259246 :  None : False : False :  Reals
     (1, 'Qт') :  None :             0.0001 :  None : False : False :  Reals
      (2, 'N') :  None :  75.03967027081069 :  None : False : False :  Reals
     (2, 'Q0') :  None : 158.64134248493644 :  None : False : False :  Reals
     (2, 'Qп') :  None : 20.000010451259246 :  None : False : False :  Reals
     (2, 'Qт') :  None :             0.0

# Блок работы с характеристиками PWL (характеристики процесса полноценные)

In [43]:
m1 = ConcreteModel()
# Задаем шкалу времени (24 часа), для которой будут проводиться расчеты
nt=24
t = range(0, nt)
m1.T1 = PWL_Block(t, df)

In [44]:
# Определяем целевую функцию
Dohod = sum((m1.T1.Vars[i,'Q0']) for i in t)
m1.O1 = Objective(expr=Dohod , sense=maximize)

In [45]:
# Используем наиболее быстрый решатель оптимизации Guroby Optimization для расчета нашей оптимизационной модели
gurobi = SolverFactory('gurobi')

In [46]:
m1.T1=ext_vars(m1.T1,Qт=df['Qт'])

['Qт']


In [47]:
# Смотрим время расчета и статус выполнения расчета модели
status = gurobi.solve(m1)
print(status)


Problem: 
- Name: x53234
  Lower bound: 3668.8112243757764
  Upper bound: 3668.8112243757773
  Number of objectives: 1
  Number of constraints: 8977
  Number of variables: 53234
  Number of binary variables: 44329
  Number of integer variables: 44329
  Number of continuous variables: 8905
  Number of nonzeros: 271912
  Sense: maximize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 9.855476379394531
  Error rc: 0
  Time: 10.801554203033447
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [19]:
m1.T1.Vars.pprint()

Vars : Size=96, Index=T1.Vars_index
    Key        : Lower : Value              : Upper : Fixed : Stale : Domain
      (0, 'N') :  None :        75.03977035 :  None : False : False :  Reals
     (0, 'Q0') :  None : 159.72861001308857 :  None : False : False :  Reals
     (0, 'Qп') :  None :               20.0 :  None : False : False :  Reals
     (0, 'Qт') :  None :                0.0 :  None : False : False :  Reals
      (1, 'N') :  None :        75.03977035 :  None : False : False :  Reals
     (1, 'Q0') :  None : 159.72861001308857 :  None : False : False :  Reals
     (1, 'Qп') :  None :               20.0 :  None : False : False :  Reals
     (1, 'Qт') :  None :                0.0 :  None : False : False :  Reals
      (2, 'N') :  None :        75.03977035 :  None : False : False :  Reals
     (2, 'Q0') :  None : 159.72861001308857 :  None : False : False :  Reals
     (2, 'Qп') :  None :               20.0 :  None : False : False :  Reals
     (2, 'Qт') :  None :                